In [ ]:
# The next cell relies on flcs having CR flags (dq value 4096), so run them through drizzle to get
# them if they dont already exist.

from drizzlepac import astrodrizzle as ad
import glob
from stsci.tools import teal

teal.unlearn('astrodrizzle')
    
files = glob.glob('*_flc.fits')
ad.AstroDrizzle(files, driz_cr_corr=True, driz_combine=True, clean=True, build=True, 
                driz_cr_snr='3.5 3.0')


In [23]:
# Flag pixels as "bad detector pixel" (dq value 4) that are within 5 pixels of a CR hit (away from the
# readout direction) AND X sigma below the image mean (where the sigma and mean here are from a Gaussian 
# fit to the sigma-clipped image data).
#
# These pixels are read out cosmic rays (ROCRs), i.e. pixels that fall during readout and therefore 
# trick the CTE correction into overcorrecting them since it thinks they fell farther from the amp
# than they actually did.

import glob
import os
from astropy.io import fits
from astropy.stats import sigma_clip
from astropy.modeling import models, fitting
import numpy as np

########################################## USER INPUTS ####################################################

# the files to use to find the ROCRs (i.e. drizzling has been done on these files so they have CR flags)
files = sorted(glob.glob('../test_data/*flc.fits'))

# the directory containing the files to add the ROCR flags to (no drizzling has been done on these files)
untouched_files_dir = '../test_data_untouched/'

# the sigma to use when determining the threshold for flagging ROCRs
sigma = 3.0

###########################################################################################################

for f in files:
    basename = os.path.basename(f)
    print('Flagging ROCRs in {} ...'.format(basename))
    untouched_file = os.path.join(untouched_files_dir, basename)
    h = fits.open(f)
    h_untouched = fits.open(untouched_file)
    
    for ext in [1,4]:
        data = h[ext].data
        dq = h[ext+2].data
        dq_untouched = h_untouched[ext+2].data
        n_y, n_x = data.shape
        
        # Find lower limit for flaggincg ROCRs
        clipped = sigma_clip(data, sigma=3, iters=5)
        d = clipped[clipped.mask==False].data
        n, bins = np.histogram(d, bins=70)
        bin_centers = (bins[:-1] + bins[1:]) / 2
        g_init = models.Gaussian1D(amplitude=n[n==max(n)], mean=bin_centers[n==max(n)], stddev=np.std(d))
        fit_g = fitting.LevMarLSQFitter()
        g = fit_g(g_init, bin_centers, n)
        thresh = g.mean.value[0] - sigma*g.stddev.value
        print('\t Threshold Ext {} = {:.3f} - {}*{:.3f} = {:.3f}'.format(ext, g.mean.value[0], sigma, 
                                                                      g.stddev.value, thresh))
    
        # Make mask of all CR hits
        cr_mask = np.zeros(dq.shape, dtype=int)
        cr_mask[dq&4096!=0] = 1

        # Flag pixels within 5 pixels of a CR hit (away from readout) that are below the threshold 
        coords = np.where(cr_mask==1)
        cr_mask_new = np.zeros(cr_mask.shape)
        for i in np.arange(len(coords[0])):
            x,y = coords[1][i], coords[0][i]

            # Get the first y-coordinate to check
            if ext==1:
                running_y = y + 1
            elif ext==4:
                running_y = y - 1
            else:
                print('extension {} not expected'.format(ext))

            # See if this coordinate has a value below the threshold
            count = 0
            while count < 5:  # stay within 5 pixels of cr hit
                if (running_y <= n_y-1) & (running_y >= 0):  # avoid going off the image y-dimension          
                    val = data[running_y, x]
                    if val < thresh:
                        cr_mask_new[running_y, x] = 1
                    if ext==1:
                        running_y += 1
                    elif ext==4:
                        running_y -= 1
                    else:
                        print('extension {} not expected'.format(ext))
                else:
                    pass

                count += 1
    
        # Add in new ROCR flags as 4 (bad detector pixel)
        dq_untouched[(dq_untouched&4==0) & (cr_mask_new==1)] += 4
        h_untouched[ext+2].data = dq_untouched
        
        # Write out ROCR flag map
        fits.writeto(f.replace('_flc.fits','_rocr_map_ext_{}.fits'.format(ext)), cr_mask_new, overwrite=True)
        print('\t # of ROCR flags in Ext {}: {}'.format(ext, len(cr_mask_new[cr_mask_new==1])))
        
    # Write out the ROCR-flagged flc file
    outname = untouched_file.replace('_flc.fits','_rocr_flagged_flc.fits')
    h_untouched.writeto(outname, overwrite=True)
    h_untouched.close()
    print('\t ROCR-flagged image saved to {}'.format(os.path.basename(outname)))


Flagging ROCRs in ie8u01ezq_flc.fits ...
	 Threshold Ext 1 = 0.061 - 3.0*5.533 = -16.537
	 # of ROCR flags in Ext 1: 30449
	 Threshold Ext 4 = 0.084 - 3.0*5.982 = -17.861
	 # of ROCR flags in Ext 4: 34854
	 ROCR-flagged image saved to ie8u01ezq_rocr_flagged_flc.fits
Flagging ROCRs in jdxfa9i7q_flc.fits ...
	 Threshold Ext 1 = 35.339 - 3.0*10.266 = 4.542
	 # of ROCR flags in Ext 1: 16428
	 Threshold Ext 4 = 34.828 - 3.0*9.967 = 4.928
	 # of ROCR flags in Ext 4: 19590
	 ROCR-flagged image saved to jdxfa9i7q_rocr_flagged_flc.fits
